# Finding Grids.
As a starter we boil down the problem to the case where neighborhoods are
made out of street grids. Now we want to find such grids in cities.
- connected
- consisting of two fixed street directions, not necessarily at $90\degree$
to each other


In [ ]:
import osmnx as ox
import networkx as nx

We start by importing a street network, make it **unidirected** and add the
streets orientations aka bearings.
Feel free to change the input graph by modifying the location string (find
fitting one with [Nominatim](https://nominatim.openstreetmap.org/ui/search.html)).
We use the example of Brooklyn, NY, because the streets consists out of
grids, which have a variety of orientations, as well as soe parts which do
not belong to a grid.

In [ ]:
G = ox.graph_from_place("Brooklyn")
G = ox.get_undirected(G)
G = ox.add_edge_bearings(G)
ox.plot_folium(G)